# What do I want to end up with?
## Tenniest teams (per year + aggregrated)
## Top 10 (20?) goats of all time (by average score) (by apparatus?)
## Bubble maps x axis year, y axis team, size = no. 10s (colour/pie apparatus if poss?)
## Avg score over time (colour by team)

# 1 Set up the environment

In [1]:
!pip install -r ../requirements.txt

In [2]:
import os
import json
import requests
import sqlite3
from tqdm.notebook import tqdm, trange
tqdm.pandas()
import numpy as np
import pandas as pd 
from sqlalchemy import create_engine
from lets_plot import * # This imports all of ggplot2's functions
LetsPlot.setup_html()
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output


%load_ext sql
%config SqlMagic.autocommit=True

from pprint import pprint

## 1.1 Connect to the gymternet database

In [3]:
%sql sqlite:///../data/clean/gymternet.db --alias gymternet 
engine = create_engine('sqlite:///../data/clean/gymternet.db')

Connecting to 'gymternet'

# 2 Exploratory data analysis

## 2.1 The tenniest apparatus
Which apparatus (vault, uneven bars, balance beam or floor exercise) attract the most 10s from the judges? Has it changed over time?

Intuitively, one would assume that vault would attract the fewest deductions; gymnasts are only performing one skill, so there are fewer opportunities to make mistakes.

However, my anecdotal observation as a watcher of college gymnastics is that the judges in this competition are fairly lenient; hesitancy on beam or short handstands on bars might not incur the deduction they would in other leagues. However, they are quite strict on landings - that is, if a gymnast doesn't perfectly stick their landing, they will incur a deduction. Given vault's landing difficulty, does this even out the advantage of having to perform fewer skills?

### 2.1.1 Retrieve the data from the database

In [4]:
%%sql gymternet

-- LEFT JOIN with aggregated row at the bottom
SELECT 
    SUM(r.vt_score = 10.0) AS 'Vault',
    SUM(r.ub_score = 10.0) AS 'Uneven Bars',
    SUM(r.bb_score = 10.0) AS 'Balance Beam',
    SUM(r.fx_score = 10.0) AS 'Floor Exercise',
    m.year AS 'Season'
FROM gymnast_results AS r
LEFT JOIN meets AS m
ON m.meet_id = r.meet_id
GROUP BY m.year

Vault,Uneven Bars,Balance Beam,Floor Exercise,Season
34,32,2,7,2015
12,8,16,28,2016
22,26,35,16,2017
10,51,53,24,2018
31,38,8,56,2019
28,10,32,4,2020
50,44,20,21,2021
59,46,38,77,2022
88,81,126,64,2023
45,56,69,103,2024


### 2.1.2 Import the data into a dataframe

In [5]:
# Export the above query to a new df
tenniest_apparatus_query = """
SELECT 
    SUM(r.vt_score = 10.0) AS 'Vault',
    SUM(r.ub_score = 10.0) AS 'Uneven Bars',
    SUM(r.bb_score = 10.0) AS 'Balance Beam',
    SUM(r.fx_score = 10.0) AS 'Floor Exercise',
    m.year AS 'Season'
FROM gymnast_results AS r
LEFT JOIN meets AS m
ON m.meet_id = r.meet_id
GROUP BY m.year;
"""

# Execute the query and store the result in a DataFrame
tenniest_apparatus_df = pd.read_sql_query(tenniest_apparatus_query, engine)

# Preview the df
tenniest_apparatus_df

,Vault,Uneven Bars,Balance Beam,Floor Exercise,Season
0,34,32,2,7,2015
1,12,8,16,28,2016
2,22,26,35,16,2017
3,10,51,53,24,2018
4,31,38,8,56,2019
5,28,10,32,4,2020
6,50,44,20,21,2021
7,59,46,38,77,2022
8,88,81,126,64,2023
9,45,56,69,103,2024


### 2.1.3 Prepare the data for plotting

We want this table to look slightly different, so that it's easier to read by Plotly.

New layout should look like:
| **Apparatus**    | **Number of 10s** | **Season** |
|------------------|-------------------|------------|
| 'Vault'          | 34                | 2015       |
| 'Uneven Bars'    | 32                | 2015       |
| 'Balance Beam'   | 2                 | 2015       |
| 'Floor Exercise' | 7                 | 2015       |
| 'Total'          | 75                | 2015       |

etc.


In [6]:
# Melt the DataFrame
tenniest_apparatus_per_year = pd.melt(tenniest_apparatus_df, id_vars=['Season'], var_name='Apparatus', value_name='No. of Tens')

# Preview the melted DataFrame
tenniest_apparatus_per_year.head()

,Season,Apparatus,No. of Tens
0,2015,Vault,34
1,2016,Vault,12
2,2017,Vault,22
3,2018,Vault,10
4,2019,Vault,31


### 2.1.4 Prepare the plots

I want to explore how many 10s have been awarded across each of the apparatus in total across the last 10 years.

To visualise this, I want a stacked bar chart, with apparatus across the x-axis, number of 10s on the y-axis and for each bar to be segmented by year.

I also want to be fairly specific about my colour, field and font choices, as I'm preparing all the upcoming charts for publication on a website, and I want them to look as though they belong together.

#### 2.1.4.1 Tenniest apparatus ever

In [7]:
# A bar chart showing the aggregated number of 10s per apparatus across the years

tenniest_apparatus_ever = (
        ggplot(tenniest_apparatus_per_year, aes(x='Apparatus', y='No. of Tens')) + 
            geom_bar(aes(group='Season', fill='Season'), 
                stat='identity', 
                alpha=.8,
                size=0.2) +
            ggtitle('Which apparatus attracts the most 10s in NCAA gymnastics?') +
            scale_fill_viridis() +
            scale_fill_discrete() +
            theme(
                axis_title = element_text(size = 12, family='Helvetica'),
                axis_text = element_text(size = 12, family='Helvetica'),
                legend_position='bottom',
                legend_title = element_text(size = 12, family='Helvetica'),
                legend_text = element_text(size = 10, family = 'Helvetica')
            )      
)

# Export the plot to html file
ggsave(tenniest_apparatus_ever, "../docs/figures/tenniest_apparatus_ever.html")

# Show the plot
tenniest_apparatus_ever

By the looks of the above plot, it seems like there isn't that much difference in the likelihood of scoring a 10 on any particular apparatus, although there is a slight advantage on Floor Exercise. 

The sizes of the slices, however, tell a different story. It suggests there are trends; in some seasons it is easier to achieve perfection on one apparatus and in others another.

It would be interesting to explore how these trends change from year to year in some sort of amusing animated plot.

#### 2.1.4.2 Tenniest apparatus per year

In [8]:
# Making an animated plot to show the number of 10s scored on each apparatus over the years

# Sample distinct colors from the Viridis color scale
num_colors = len(tenniest_apparatus_per_year['Apparatus'].unique())
viridis_colors = px.colors.sample_colorscale(px.colors.sequential.Viridis, [i/num_colors for i in range(num_colors)])

tenniest_apparatus_py = px.bar(tenniest_apparatus_per_year, 
                x="Apparatus", 
                y="No. of Tens", 
                animation_frame="Season",       
                color="Apparatus", 
                hover_name="Apparatus",
                range_y=[0, tenniest_apparatus_per_year["No. of Tens"].max()], # Set the y-axis range
                color_discrete_sequence=viridis_colors,
                opacity=0.8
            )

# Customize the layout
tenniest_apparatus_py.update_layout(
    title="Number of 10s Scored on Each Apparatus Over the Years",
    title_font=dict(size=12, family='Helvetica', color='black'),
    xaxis_title="Apparatus",
    xaxis_title_font=dict(size=12, family='Helvetica', color='black'),
    yaxis_title="No. of Tens",
    yaxis_title_font=dict(size=12, family='Helvetica', color='black'),
    legend_title="Apparatus",
    legend_title_font=dict(size=12, family='Helvetica', color='black'),
    font=dict(size=10, family='Helvetica', color='black'),
    plot_bgcolor='white',  # Set plot background to white
    paper_bgcolor='white',  # Set paper background to white
    xaxis=dict(
        gridcolor='#EEEEEE'  # Set x-axis grid lines to light grey
    ),
    yaxis=dict(
        gridcolor='#EEEEEE'  # Set y-axis grid lines to light grey
    ),
    legend=dict(
        orientation="h",  #horizontal legend
        yanchor="bottom",  
        y=-1,  
        xanchor="center",  
        x=0.5  
    )
)
# Export the plot to html file
tenniest_apparatus_py.write_html("../docs/figures/tenniest_apparatus_per_year.html")

# Show the plot
tenniest_apparatus_py

## 2.2 The tenniest teams

Ok, but this is a competition, isn't it? Which *teams* have been the most successful in achieving tens over the years? Has it changed over time?

I'd like to run a similar analysis to the above, but this time looking at which teams are the most likely to score a 10. 

In recent years, Oklahoma has been the most dominant team, with flashes of brilliance from UCLA, LSU and Michigan. Florida's team is currently stacked with former Olympians and National Team members, so one would assume they would also be highly ranked on this metric.

### 2.2.1 Retrieve the data from the database

In [9]:
%%sql --alias gymternet

SELECT 
    SUM(r.vt_score = 10.0) AS 'Vault',
    SUM(r.ub_score = 10.0) AS 'Uneven Bars',
    SUM(r.bb_score = 10.0) AS 'Balance Beam',
    SUM(r.fx_score = 10.0) AS 'Floor Exercise',
    SUM(r.vt_score = 10.0) + SUM(r.ub_score = 10.0) + SUM(r.bb_score = 10.0) + SUM(r.fx_score = 10.0) AS 'Total 10s',
    g.team_id AS 'team_id',
    t.team_name AS 'Team',
    m.year AS 'Season'
FROM gymnast_results AS r
LEFT JOIN gymnasts AS g
ON g.gymnast_id = r.gymnast_id
LEFT JOIN teams as t
ON t.team_id = g.team_id
LEFT JOIN meets as m
ON m.meet_id = r.meet_id
GROUP BY t.team_name, r.meet_id;

Running query in 'gymternet'

Vault,Uneven Bars,Balance Beam,Floor Exercise,Total 10s,team_id,Team,Season
0,0,0,0,0,1,Air Force,2015
0,0,0,0,0,1,Air Force,2015
0,0,0,0,0,1,Air Force,2015
0,0,0,0,0,1,Air Force,2015
0,0,0,0,0,1,Air Force,2015
0,0,0,0,0,1,Air Force,2015
0,0,0,0,0,1,Air Force,2015
0,0,0,0,0,1,Air Force,2015
0,0,0,0,0,1,Air Force,2015
0,0,0,0,0,1,Air Force,2015


### 2.2.2 Import the data into a dataframe

In [10]:
# Export the above query to a new df
tenniest_teams_query = """
SELECT 
    SUM(r.vt_score = 10.0) AS 'Vault',
    SUM(r.ub_score = 10.0) AS 'Uneven Bars',
    SUM(r.bb_score = 10.0) AS 'Balance Beam',
    SUM(r.fx_score = 10.0) AS 'Floor Exercise',
    SUM(r.vt_score = 10.0) + SUM(r.ub_score = 10.0) + SUM(r.bb_score = 10.0) + SUM(r.fx_score = 10.0) AS 'Total 10s',
    g.team_id AS 'team_id',
    t.team_name AS 'Team',
    m.year AS 'Season'
FROM gymnast_results AS r
LEFT JOIN gymnasts AS g
ON g.gymnast_id = r.gymnast_id
LEFT JOIN teams as t
ON t.team_id = g.team_id
LEFT JOIN meets as m
ON m.meet_id = r.meet_id
GROUP BY t.team_name, r.meet_id;
"""

# Execute the query and store the result in a DataFrame
tenniest_teams_df = pd.read_sql_query(tenniest_teams_query, engine)

# Drop the now irrelevant team_id column
tenniest_teams_df = tenniest_teams_df.drop(columns=['team_id'])

# Preview the df
tenniest_teams_df.head()

,Vault,Uneven Bars,Balance Beam,Floor Exercise,Total 10s,Team,Season
0,0.0,0.0,0.0,0.0,0.0,Air Force,2015
1,0.0,0.0,0.0,0.0,0.0,Air Force,2015
2,0.0,0.0,0.0,0.0,0.0,Air Force,2015
3,0.0,0.0,0.0,0.0,0.0,Air Force,2015
4,0.0,0.0,0.0,0.0,0.0,Air Force,2015


### 2.2.3 Prepare the data for plotting

I'd like to ignore any team that has never gotten a 10, and then create some visualisations that tell me:

- Which teams have gotten the most tens in total over the past 10 years
- Which teams have dominated (in terms of number of 10s earned) across each apparatus
- How these two analyses have changed over the past 10 years.

In [11]:
# Let's remove the teams that have never gotten a 10
grouped_teams_df = tenniest_teams_df.groupby(['Team']).sum().reset_index()

# Any let's drop the irrelevant columns
grouped_teams_df = grouped_teams_df.drop(columns = ['Season'])

# Preview the new df
grouped_teams_df.head()

,Team,Vault,Uneven Bars,Balance Beam,Floor Exercise,Total 10s
0,Air Force,0.0,0.0,0.0,0.0,0.0
1,Alabama,8.0,12.0,18.0,6.0,44.0
2,Alaska,0.0,0.0,0.0,0.0,0.0
3,Arizona,0.0,0.0,0.0,0.0,0.0
4,Arizona State,0.0,0.0,0.0,0.0,0.0


In [12]:
# Drop rows where total 10s == 0
grouped_teams_df = grouped_teams_df[grouped_teams_df['Total 10s'] != 0]

grouped_teams_df = grouped_teams_df.sort_values(by='Team', ascending=True)

# Check how many we have
grouped_teams_df.shape

(29, 6)

In [13]:
# Sort the grouped_teams df by number of overall tens
grouped_teams_df = grouped_teams_df.sort_values(by='Total 10s', ascending=True).reset_index()

#Preview the df
grouped_teams_df.head()

,index,Team,Vault,Uneven Bars,Balance Beam,Floor Exercise,Total 10s
0,63,Southern Utah,0.0,0.0,2.0,0.0,2.0
1,34,Kent State,0.0,0.0,0.0,4.0,4.0
2,52,Penn State,0.0,4.0,0.0,0.0,4.0
3,57,S.E. Missouri,0.0,4.0,0.0,0.0,4.0
4,62,Southern Conn.,0.0,4.0,0.0,0.0,4.0


That's looking good! The `grouped_teams` DataFrame will feed the plot that analyses the Tenniest Teams of All Time*.

*For the purposes of this analysis, time began in 2015.

Now I need to create a longer DataFrame that can facet along the apparatus.

In [14]:
# Melt the dataframe so that the values in the 'Vault' 'Uneven Bars' 'Balance Beam' and 'Floor Exercise' columns are all in a new column called 'Apparatus'
tenniest_teams_per_apparatus = pd.melt(grouped_teams_df, id_vars=['Team'], value_vars=['Vault', 'Uneven Bars', 'Balance Beam', 'Floor Exercise'], var_name='Apparatus', value_name='No. of Tens')

# Preview the df
tenniest_teams_per_apparatus.head()

,Team,Apparatus,No. of Tens
0,Southern Utah,Vault,0.0
1,Kent State,Vault,0.0
2,Penn State,Vault,0.0
3,S.E. Missouri,Vault,0.0
4,Southern Conn.,Vault,0.0


These dataframes lose information about the Season and later on, I might like to look at Teams' performances over time.

In [15]:
# Calculate sum of number of 10s for each apparatus for each team for each season
tenniest_teams_over_time = tenniest_teams_df.groupby(['Team', 'Season']).sum().reset_index()

# Preview the df
tenniest_teams_over_time.head()

,Team,Season,Vault,Uneven Bars,Balance Beam,Floor Exercise,Total 10s
0,Air Force,2015,0.0,0.0,0.0,0.0,0.0
1,Air Force,2016,0.0,0.0,0.0,0.0,0.0
2,Air Force,2017,0.0,0.0,0.0,0.0,0.0
3,Air Force,2018,0.0,0.0,0.0,0.0,0.0
4,Air Force,2019,0.0,0.0,0.0,0.0,0.0


In [16]:
# Remove the teams that have never received a 10 (these have already been filtered out in the grouped_df)
teams = grouped_teams_df['Team'].tolist()

# Delete entries if Team is in teams
tenniest_teams_over_time = tenniest_teams_over_time[tenniest_teams_over_time['Team'].isin(teams)]

# Preview the df
tenniest_teams_over_time.head()

,Team,Season,Vault,Uneven Bars,Balance Beam,Floor Exercise,Total 10s
10,Alabama,2015,0.0,0.0,0.0,0.0,0.0
11,Alabama,2016,2.0,0.0,0.0,0.0,2.0
12,Alabama,2017,0.0,0.0,0.0,2.0,2.0
13,Alabama,2018,0.0,0.0,0.0,0.0,0.0
14,Alabama,2019,0.0,0.0,0.0,0.0,0.0


In [17]:
# Melt this df for easier analysis
tenniest_teams_over_time = tenniest_teams_over_time.melt(id_vars=["Season", "Team"], value_vars=["Vault", "Uneven Bars", "Balance Beam", "Floor Exercise", "Total 10s"], var_name="Apparatus", value_name="No. of 10s")

# Update rows that say "Total 10s" to say "Total"
tenniest_teams_over_time['Apparatus'] = tenniest_teams_over_time['Apparatus'].replace('Total 10s', 'Total')

tenniest_teams_over_time

,Season,Team,Apparatus,No. of 10s
0,2015,Alabama,Vault,0.0
1,2016,Alabama,Vault,2.0
2,2017,Alabama,Vault,0.0
3,2018,Alabama,Vault,0.0
4,2019,Alabama,Vault,0.0
...,...,...,...,...
1440,2020,Washington,Total,4.0
1441,2021,Washington,Total,0.0
1442,2022,Washington,Total,0.0
1443,2023,Washington,Total,0.0


### 2.2.4 Prepare the plots

First off, I want to see a bar chart showing the aggregated number of 10s per apparatus across the years.

#### 2.2.4.1 Tenniest teams ever

In [18]:
# A bar chart showing the aggregated number of 10s per apparatus across the years

tenniest_teams_ever = (
    ggplot(grouped_teams_df) +
    geom_bar(aes(x='Total 10s', y='Team', fill='Total 10s'), 
            tooltips=layer_tooltips()
            .line(('@|@Team'))
            .line(('Total 10s|@{Total 10s}'))
            .format('@|<b>{}</b>', '@|{}'),
            stat='identity', alpha=0.8) +
    ggtitle('Which team has had the most number of 10s?') +
    scale_fill_viridis() +
    theme(
        axis_title=element_text(size=12, family='Helvetica'),
        axis_text=element_text(size=12, family='Helvetica'),
        legend_position='bottom',
        legend_title=element_text(size=12, family='Helvetica'),
        legend_text=element_text(size=10, family='Helvetica')
    ) +
    ggsize(1200, 1200)
)


# Export the plot to html file
ggsave(tenniest_teams_ever, "../docs/figures/tenniest_teams_ever.html")

# Show the plot
tenniest_teams_ever

#### 2.2.4.2 Tenniest teams per apparatus

Let's see how this data breaks down over each apparatus? Is it true that Utah are beam queens? Is it true that UCLA can't be beaten on floor? I know *I* love the perfect handstands and extended lines of Cal's bars, but do the judges? How much are Suki Pfister's perfect vaults contributing to Ball State's overall 10 performance?

In [19]:
tenniest_teams_per_event = (
    ggplot(tenniest_teams_per_apparatus) +
    geom_bar(aes(x="No. of Tens", y="Team", fill='No. of Tens'), 
            tooltips=layer_tooltips()
            .line(('@|@Team'))
            .line(('Total 10s|@{No. of Tens}'))
            .line('@|@Apparatus')
            .format('@|<b>{}</b>', '@|{}'),
            stat='identity',
            alpha=0.8) +
    facet_wrap('Apparatus') +
    ggtitle('Total Number of 10s by Team and Apparatus') +
    scale_fill_viridis() +
    theme(
        axis_title=element_text(size=12, family='Helvetica'),
        axis_text=element_text(size=12, family='Helvetica'),
        legend_position='bottom',
        legend_title=element_text(size=12, family='Helvetica'),
        legend_text=element_text(size=10, family='Helvetica')
    ) +
    ggsize(1200, 1800)
)

# Export the plot to html file
ggsave(tenniest_teams_per_event, "../docs/figures/tenniest_teams_per_apparatus.html")

# Show the plot
tenniest_teams_per_event

The criminal underrecognition of Cal's bars. The absolute disrespect 😤.

#### 2.2.4.3 Tenniest teams per apparatus, over time

We're about to get ambitious. I'd like to be able to compare, head-to-head any teams that I like on any apparatus that I like. What if we have a series of user-controlled filters?

Big shout-out to co-pilot for co-authoring this section.

In [20]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H4('Tenniest teams per apparatus, over time', style={'font-family': 'Helvetica'}),
    
    # Dropdown for selecting teams
    dcc.Dropdown(
        id="dropdown",
        options=[{"label": team, "value": team} for team in tenniest_teams_over_time["Team"].unique()],
        value=tenniest_teams_over_time["Team"].unique().tolist(),  # Default to showing all teams
        multi=True,
        style={'font-family': 'Helvetica', 'font-size': '10px'}  # Font styling for dropdown text
    ),

    # Checklist for selecting events (apparatus)
    dcc.Checklist(
        id="event_checklist",
        options=[
            {"label": "Vault", "value": "Vault"},
            {"label": "Uneven Bars", "value": "Uneven Bars"},
            {"label": "Balance Beam", "value": "Balance Beam"},
            {"label": "Floor Exercise", "value": "Floor Exercise"},
            {"label": "Total", "value": "Total"}
        ],
        value=["Vault", "Uneven Bars", "Balance Beam", "Floor Exercise", "Total"],  # Default to showing all events
        inline=True
    ),

    # Graph component to display the plot
    dcc.Graph(id="graph")
])

# Define callback to update the plot based on user inputs
@app.callback(
    Output("graph", "figure"),
    [Input("dropdown", "value"),
     Input("event_checklist", "value")]
)
def update_plot(selected_teams, selected_events):

    # Filter the dataset based on selected teams and events
    filtered_df = tenniest_teams_over_time[
        (tenniest_teams_over_time["Team"].isin(selected_teams)) &
        (tenniest_teams_over_time["Apparatus"].isin(selected_events))
    ]
    
    # Create a line plot
    tenniest_teams_per_apparatus_per_year = px.line(filtered_df, 
                  x="Season", 
                  y="No. of 10s", 
                  color="Team",
                  line_dash="Apparatus",  # Different line styles for each event
                  labels={"No. of 10s": "No. of 10s", "Apparatus": "Apparatus"})
    
    # Update layout for formatting
    tenniest_teams_per_apparatus_per_year.update_layout(
        title='Tenniest teams per apparatus, per year',
        xaxis_title='Season',
        yaxis_title='No. of 10s',
        plot_bgcolor='white',  
        font=dict(
            family="Helvetica",  
            size=12,  
            color='black'  
        ),
        legend=dict(
            font=dict(
                size=10  
            )
        )
    )
    
    # Configure axes appearance
    tenniest_teams_per_apparatus_per_year.update_yaxes(showgrid=True, gridcolor='#DDDDDD', color='black', title_font=dict(color='black'), tickfont=dict(color='black'))
    tenniest_teams_per_apparatus_per_year.update_xaxes(color='black', title_font=dict(color='black'), tickfont=dict(color='black'))
    
    # Export the figure to an HTML file
    tenniest_teams_per_apparatus_per_year.write_html("../docs/figures/tenniest_eams_per_apparatus_per_year.html")

    return tenniest_teams_per_apparatus_per_year

# Run the app server in debug mode
if __name__ == '__main__':
    app.run_server(debug=True)

## 2.3 Ten of the top 10 GOATs of all time

Do you think I'm only interested in 10s? Children focus on 10s. I'm interested in the truth. I'm interested in what matters. 10s are shiny, certainly, but any good person-who-can-do-basic-mathematics can see that a gymnast who gets a 10 one week and then a 5 the next week is not as useful as a gymnast that gets a 9.9 week after week.

With this in mind, let's find the gymnasts who have the highest average scores across the apparatus and across the seasons.

It's easy to do well if you never compete. For the purposes of this analysis, I am only interested in gymnasts who compete a minimum of 6 times over the course of the season (per apparatus). This will necessarily impact the data from the 2020 and 2021 seasons, which were impacted heavily by COVID restrictions.

### 2.3.1 Retrieve the data from the database

Getting the averages is straightforward enough!

In [27]:
%%sql --alias gymternet

SELECT 
    g.gymnast_id AS 'Gymnast ID',
    g.first_name AS 'First Name',
    g.last_name AS 'Last Name',
    t.team_name AS 'Team',
    AVG(r.vt_score) AS 'Vault',
    COUNT(r.vt_score) AS 'VT Count',
    AVG(r.ub_score) AS 'Uneven Bars',
    COUNT(r.ub_score) AS 'UB Count',
    AVG(r.bb_score) AS 'Balance Beam',
    COUNT(r.bb_score) AS 'BB Count',
    AVG(r.fx_score) AS 'Floor Exercise',
    COUNT(r.fx_score) AS 'FX Count',
    AVG(r.aa_score) AS 'All Around',
    COUNT(r.aa_score) AS 'AA Count',
    m.year AS 'Season'
FROM gymnast_results AS r
LEFT JOIN gymnasts AS g
ON g.gymnast_id = r.gymnast_id
LEFT JOIN teams as t
ON t.team_id = g.team_id
LEFT JOIN meets as m
ON m.meet_id = r.meet_id
GROUP BY g.gymnast_id, m.year;

Running query in 'gymternet'

First Name,Last Name,Team,Vault,VT Count,Uneven Bars,UB Count,Balance Beam,BB Count,Floor Exercise,FX Count,All Around,AA Count,Season
Kristina,Heymann,California,9.829166666666666,12,None,0,None,0,None,0,None,0,2015
Serena,Leong,California,9.839285714285714,7,None,0,9.62857142857143,7,9.65,7,None,0,2015
Serena,Leong,California,9.7625,8,None,0,None,0,9.771875,8,None,0,2016
Jordan,Widener,California,None,0,9.762162162162161,37,None,0,None,0,None,0,2015
Breanna,Hughes,Utah,9.825,2,9.756976744186046,43,None,0,9.725000000000001,8,None,0,2015
Breanna,Hughes,Utah,9.881818181818181,66,9.798484848484849,66,9.630833333333333,60,9.882575757575758,66,39.19166666666667,60,2016
Crystal,Paz,California,9.55,5,None,0,9.275,8,9.7,10,None,0,2015
Georgia,Dabritz,Utah,9.90566037735849,53,9.963068181818182,66,9.70904255319149,47,9.92253787878788,66,39.48048780487805,41,2015
Kailah,Delaney,Utah,9.885377358490565,53,None,0,9.648484848484848,33,None,0,None,0,2015
Kailah,Delaney,Utah,9.85483870967742,62,None,0,9.425833333333333,60,9.86875,8,None,0,2016


But getting the means are a giant pain with SQLite. In this case, I'll got for a maximalist approach and whittle down the data in pandas.

In [22]:
%%sql --alias gymternet

SELECT 
    g.gymnast_id AS 'Gymnast ID',
    g.first_name AS 'First Name',
    g.last_name AS 'Last Name',
    t.team_name AS 'Team',
    r.vt_score AS 'Vault',
    r.ub_score AS 'Uneven Bars',
    r.bb_score AS 'Balance Beam',
    r.fx_score AS 'Floor Exercise',
    m.year AS 'Season'
FROM gymnast_results AS r
LEFT JOIN gymnasts AS g
ON g.gymnast_id = r.gymnast_id
LEFT JOIN teams as t
ON t.team_id = g.team_id
LEFT JOIN meets as m
ON m.meet_id = r.meet_id;

Running query in 'gymternet'

First Name,Last Name,Team,Vault,Uneven Bars,Balance Beam,Floor Exercise,Season
Sierra,Ballard,LSU,None,None,9.2,9.9,2024
Sierra,Ballard,LSU,None,None,9.2,9.9,2024
Haleigh,Bryant,LSU,9.95,9.875,9.925,9.925,2024
Haleigh,Bryant,LSU,9.95,9.875,9.925,9.925,2024
Ashley,Cowan,LSU,None,9.8,None,None,2024
Ashley,Cowan,LSU,None,9.8,None,None,2024
Amari,Drayton,LSU,9.925,None,None,9.925,2024
Amari,Drayton,LSU,9.925,None,None,9.925,2024
Olivia,Dunne,LSU,None,None,None,9.875,2024
Olivia,Dunne,LSU,None,None,None,9.875,2024


### 2.3.2 Import the data into a dataframe

In [63]:
# Export the above query to a new df
average_scores_query = """
SELECT 
    g.gymnast_id AS 'Gymnast ID',
    g.first_name AS 'First Name',
    g.last_name AS 'Last Name',
    t.team_name AS 'Team',
    AVG(r.vt_score) AS 'Vault',
    COUNT(r.vt_score) AS 'VT Count',
    AVG(r.ub_score) AS 'Uneven Bars',
    COUNT(r.ub_score) AS 'UB Count',
    AVG(r.bb_score) AS 'Balance Beam',
    COUNT(r.bb_score) AS 'BB Count',
    AVG(r.fx_score) AS 'Floor Exercise',
    COUNT(r.fx_score) AS 'FX Count',
    AVG(r.aa_score) AS 'All Around',
    COUNT(r.aa_score) AS 'AA Count',
    m.year AS 'Season'
FROM gymnast_results AS r
LEFT JOIN gymnasts AS g
ON g.gymnast_id = r.gymnast_id
LEFT JOIN teams as t
ON t.team_id = g.team_id
LEFT JOIN meets as m
ON m.meet_id = r.meet_id
GROUP BY g.gymnast_id, m.year;
"""

# Execute the query and store the result in a DataFrame
average_scores_df = pd.read_sql_query(average_scores_query, engine)

# Preview the df
average_scores_df.head()

,Gymnast ID,First Name,Last Name,Team,Vault,VT Count,Uneven Bars,UB Count,Balance Beam,BB Count,Floor Exercise,FX Count,All Around,AA Count,Season
0,249,Kristina,Heymann,California,9.829167,12,NaN,0,NaN,0,NaN,0,NaN,0,2015
1,251,Serena,Leong,California,9.839286,7,NaN,0,9.628571,7,9.650000,7,NaN,0,2015
2,251,Serena,Leong,California,9.762500,8,NaN,0,NaN,0,9.771875,8,NaN,0,2016
3,258,Jordan,Widener,California,NaN,0,9.762162,37,NaN,0,NaN,0,NaN,0,2015
4,1158,Breanna,Hughes,Utah,9.825000,2,9.756977,43,NaN,0,9.725000,8,NaN,0,2015


In [59]:
# Export the above query to a new df
median_scores_query = """
SELECT 
    g.gymnast_id AS 'Gymnast ID',
    g.first_name AS 'First Name',
    g.last_name AS 'Last Name',
    t.team_name AS 'Team',
    r.vt_score AS 'Vault',
    r.ub_score AS 'Uneven Bars',
    r.bb_score AS 'Balance Beam',
    r.fx_score AS 'Floor Exercise',
    m.year AS 'Season'
FROM gymnast_results AS r
LEFT JOIN gymnasts AS g
ON g.gymnast_id = r.gymnast_id
LEFT JOIN teams as t
ON t.team_id = g.team_id
LEFT JOIN meets as m
ON m.meet_id = r.meet_id;
"""

# Execute the query and store the result in a DataFrame
median_scores_df = pd.read_sql_query(median_scores_query, engine)

# Preview the df
median_scores_df.head()

,Gymnast ID,First Name,Last Name,Team,Vault,Uneven Bars,Balance Beam,Floor Exercise,Season
0,30950,Sierra,Ballard,LSU,NaN,NaN,9.200,9.900,2024
1,30950,Sierra,Ballard,LSU,NaN,NaN,9.200,9.900,2024
2,30952,Haleigh,Bryant,LSU,9.95,9.875,9.925,9.925,2024
3,30952,Haleigh,Bryant,LSU,9.95,9.875,9.925,9.925,2024
4,31947,Ashley,Cowan,LSU,NaN,9.800,NaN,NaN,2024


### 2.3.3 Prepare the data for plotting

In each season, a GOAT candidate must have competed at least 6 times per apparatus (or All Around) of candidacy.

I could go ahead and exclude all rows where `VT Count`, `UB Count`, `BB Count`, `FX Count` AND `AA Count` ALL are less than 6, but I think it's probably going to be easier to work with dataframes specific to the apparatus. This gives me an opportunity to limit the DataFrames to the top-20 in each apparatus as well.

In [70]:
# First up, the VT DF

# Drop any rows where a gymnast has performed fewer than 6 vaults
vault_average_df = average_scores_df[(average_scores_df['VT Count'] >= 6)]

# Sort by average vault score in descending order
vault_average_df = vault_average_df.sort_values(by='Vault', ascending=False)

# Get the first 20 unique values in the 'Gymnast ID' column
top_20_ave_vaulters = vault_average_df['Gymnast ID'].drop_duplicates().head(20).tolist()

# Filter the original DataFrame to include all rows with these unique values
vault_average_df = vault_average_df[vault_average_df['Gymnast ID'].isin(top_20_ave_vaulters)]

# Preview the df
vault_average_df.head()

,Gymnast ID,First Name,Last Name,Team,Vault,VT Count,Uneven Bars,UB Count,Balance Beam,BB Count,Floor Exercise,FX Count,All Around,AA Count,Season
6941,30055,Olivia,Trautman,Oklahoma,9.956818,11,9.867857,21,9.853846,13,NaN,0,NaN,0,2020
6944,30055,Olivia,Trautman,Oklahoma,9.954717,53,9.916509,53,9.851866,67,NaN,0,NaN,0,2023
9166,30952,Haleigh,Bryant,LSU,9.948529,51,9.857843,51,9.676163,43,9.712255,51,39.205233,43,2021
4640,24714,Maggie,Nichols,Oklahoma,9.948148,27,9.953448,29,9.858333,27,9.938889,18,39.738889,18,2020
1215,23070,Elizabeth,Price,Stanford,9.938837,62,9.828302,53,9.900000,6,9.878947,19,NaN,0,2015


In [74]:
# Next, the UB DF

# Drop any rows where a gymnast has performed fewer than 6 bars routines
bars_average_df = average_scores_df[(average_scores_df['UB Count'] >= 6)]

# Sort by average bars score in descending order
bars_average_df = bars_average_df.sort_values(by='Uneven Bars', ascending=False)

# Get the first 20 unique values in the 'Gymnast ID' column
top_20_ave_barsers = bars_average_df['Gymnast ID'].drop_duplicates().head(20).tolist()

# Filter the original DataFrame to include all rows with these unique values
bars_average_df = bars_average_df[bars_average_df['Gymnast ID'].isin(top_20_ave_barsers)]

# Preview the df
bars_average_df.head()

,Gymnast ID,First Name,Last Name,Team,Vault,VT Count,Uneven Bars,UB Count,Balance Beam,BB Count,Floor Exercise,FX Count,All Around,AA Count,Season
10870,31789,Jordan,Chiles,UCLA,9.906818,77,9.977597,77,9.817532,77,9.957143,77,39.659091,77,2023
7,2505,Georgia,Dabritz,Utah,9.905660,53,9.963068,66,9.709043,47,9.922538,66,39.480488,41,2015
9904,31267,Sunisa,Lee,Auburn,9.900000,20,9.956818,22,9.897727,22,9.894444,18,39.705556,18,2023
569,22717,Bridget,Sloan,Florida,9.937245,49,9.955455,55,9.893056,36,9.825833,30,39.612500,22,2015
4639,24714,Maggie,Nichols,Oklahoma,9.936250,40,9.955000,60,9.916379,58,9.925000,16,39.721875,16,2019


In [75]:
# Next, the BB DF

# Drop any rows where a gymnast has performed fewer than 6 beam routines
beam_average_df = average_scores_df[(average_scores_df['BB Count'] >= 6)]

# Sort by average beam score in descending order
beam_average_df = beam_average_df.sort_values(by='Balance Beam', ascending=False)

# Get the first 20 unique values in the 'Gymnast ID' column
top_20_ave_beamers = beam_average_df['Gymnast ID'].drop_duplicates().head(20).tolist()

# Filter the original DataFrame to include all rows with these unique values
beam_average_df = beam_average_df[beam_average_df['Gymnast ID'].isin(top_20_ave_beamers)]

# Preview the df
beam_average_df.head()

,Gymnast ID,First Name,Last Name,Team,Vault,VT Count,Uneven Bars,UB Count,Balance Beam,BB Count,Floor Exercise,FX Count,All Around,AA Count,Season
8151,30545,Maile,O'Keefe,Utah,9.845000,55,9.900649,77,9.967532,77,9.917063,63,39.638182,55,2023
4638,24714,Maggie,Nichols,Oklahoma,9.920000,60,9.947177,62,9.954032,62,9.944960,62,39.770602,54,2018
8741,30786,Luisa,Blanco,Alabama,9.866667,48,9.924185,46,9.950815,46,9.932143,42,39.704762,42,2021
1359,23140,Peng-Peng,Lee,UCLA,NaN,0,9.843939,66,9.949414,64,NaN,0,NaN,0,2018
7949,30466,Sienna,Schreiber,Missouri,9.837924,59,9.784322,59,9.945339,59,9.855720,59,39.423305,59,2022


In [76]:
# Next, the FX DF

# Drop any rows where a gymnast has performed fewer than 6 floor routines
floor_average_df = average_scores_df[(average_scores_df['FX Count'] >= 6)]

# Sort by average beam score in descending order
floor_average_df = floor_average_df.sort_values(by='Floor Exercise', ascending=False)

# Get the first 20 unique values in the 'Gymnast ID' column
top_20_ave_floorers = floor_average_df['Gymnast ID'].drop_duplicates().head(20).tolist()

# Filter the original DataFrame to include all rows with these unique values
floor_average_df = floor_average_df[floor_average_df['Gymnast ID'].isin(top_20_ave_floorers)]

# Preview the df
floor_average_df.head()

,Gymnast ID,First Name,Last Name,Team,Vault,VT Count,Uneven Bars,UB Count,Balance Beam,BB Count,Floor Exercise,FX Count,All Around,AA Count,Season
6643,29946,Trinity,Thomas,Florida,9.930952,63,9.949603,63,9.943160,53,9.992857,49,39.830053,47,2022
3513,24237,Katelyn,Ohashi,UCLA,9.787500,4,NaN,0,9.927381,63,9.964796,49,NaN,0,2019
6641,29946,Trinity,Thomas,Florida,9.876316,19,9.847368,19,9.928947,19,9.959211,19,39.611842,19,2020
9226,30973,Mya,Hooten,Minnesota,9.894318,66,9.836742,66,NaN,0,9.957787,61,NaN,0,2022
6940,30055,Olivia,Trautman,Oklahoma,9.879508,61,9.875000,2,9.873305,59,9.957787,61,39.725000,2,2019


In [80]:
# Finally, the AA DF

# Drop any rows where a gymnast has performed fewer than 6 floor routines
aa_average_df = average_scores_df[(average_scores_df['AA Count'] >= 6)]

# Sort by average beam score in descending order
aa_average_df = aa_average_df.sort_values(by='All Around', ascending=False)

# Get the first 20 unique values in the 'Gymnast ID' column
top_20_ave_aaers = aa_average_df['Gymnast ID'].drop_duplicates().head(20).tolist()

# Filter the original DataFrame to include all rows with these unique values
aa_average_df = aa_average_df[aa_average_df['Gymnast ID'].isin(top_20_ave_aaers)]

# Preview the df
aa_average_df.head(60)

,Gymnast ID,First Name,Last Name,Team,Vault,VT Count,Uneven Bars,UB Count,Balance Beam,BB Count,Floor Exercise,FX Count,All Around,AA Count,Season
6643,29946,Trinity,Thomas,Florida,9.930952,63,9.949603,63,9.943160,53,9.992857,49,39.830053,47,2022
4638,24714,Maggie,Nichols,Oklahoma,9.920000,60,9.947177,62,9.954032,62,9.944960,62,39.770602,54,2018
9332,31018,Jade,Carey,Oregon State,9.928571,56,9.942857,56,9.934375,56,9.949554,56,39.755357,56,2022
6644,29946,Trinity,Thomas,Florida,9.908036,56,9.951786,56,9.915556,45,8.136585,41,39.741667,24,2023
4640,24714,Maggie,Nichols,Oklahoma,9.948148,27,9.953448,29,9.858333,27,9.938889,18,39.738889,18,2020
9169,30952,Haleigh,Bryant,LSU,9.935507,69,9.922826,69,9.930435,69,9.943654,65,39.734038,65,2024
4639,24714,Maggie,Nichols,Oklahoma,9.936250,40,9.955000,60,9.916379,58,9.925000,16,39.721875,16,2019
4830,24802,Kyla,Ross,UCLA,9.905000,25,9.953704,27,9.930556,27,9.932609,23,39.717391,23,2020
9904,31267,Sunisa,Lee,Auburn,9.900000,20,9.956818,22,9.897727,22,9.894444,18,39.705556,18,2023
8741,30786,Luisa,Blanco,Alabama,9.866667,48,9.924185,46,9.950815,46,9.932143,42,39.704762,42,2021


# Who got the highest average/median of all time
# Who got the highest average/median each year

# Which teams own the most goats?

### 2.3.4 Prepare the plots